<a href="https://colab.research.google.com/github/Sofia-1234567/Forensic-Research-Project-2021/blob/main/Tweets_Collection_for_FRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [134]:
#Twitter properties file contains the Twitter accesstoken, accesstokensecret,apikey and apisecretkey. 
!ls '/content/drive/My Drive/Colab Notebooks/twitterproperties.txt'

'/content/drive/My Drive/Colab Notebooks/twitterproperties.txt'


In [135]:
#!pip install tweepy

In [136]:
#Tweepy is a Python library that allows us to access Twitter API. 
import tweepy

#Using pandas library, a library for data manipulation and analysis to build a dataframe for the collected tweets
import pandas as pd

In [137]:
#Reads the properties file from the Google Drive

!pip install ConfigParser

In [138]:
#In the properties file, the tokens and secret keys are under a section called 'Twitter' which configparser reads. 
import pandas as pd
import configparser
config = configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/twitterproperties.txt')

print(config.sections());

['twitter']


In [139]:
import tweepy as tw
print(tw.__version__)

3.10.0


In [140]:
#Assigns the contents of the token and secret keys from the Twitter section of the properties file to the variables. 
accesstoken=config.get('twitter', 'accesstoken')
accesstokensecret=config.get('twitter', 'accesstokensecret')
apikey=config.get('twitter', 'apikey')
apisecretkey=config.get('twitter', 'apisecretkey')

In [141]:
#Tweepy AuthHandler authorises the keys and tokens to be accessed 
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)

#calling on auth and applying a wait limit so the server is not constantly pulling tweets (prevents crashing)
#when the treshold of collected tweets is met, it will refresh in another 15 minutes to pull the next set of tweets
api = tw.API(auth, wait_on_rate_limit=True)

In [142]:
#searches for words, and multiple words in one tweet using the OR function
#apply search filters
search_words = 'megan markle OR hate OR hate OR useless' '-filter:retweets AND filter:twimg'

#specify date we want the tweets from
date_since = "2021-01-01"

#tells the tweepy cursor that we want to perform an API search of search words, with language of the tweets being english, and to pull 300 tweets at a time. 
tweets = tw.Cursor(api.search,
                   q=search_words,
                   lang="en",
                   since=date_since, tweet_mode='extended', include_entities=True).items(300) 

In [143]:
#the items that will be extracted per tweet are: unique tweet identifier (as an integer), the full text of the tweet, the date tweet was created, and the URL to the image in attached to the tweet 
tweet_details = [[tweet.id, tweet.full_text, tweet.created_at, tweet.extended_entities['media'][0]['media_url_https']] for tweet in tweets]


In [148]:
#creating a dataframe of tweet_details
#column headings are: 
tweet_df = pd.DataFrame(data=tweet_details, columns=['tweet_id','tweet_txt', 'created_at', 'tweet_img_URL'])

#sets the maximum width in characters of a column. When the column overflows, a "..." placeholder is embedded in the output. 
pd.set_option('max_colwidth', 400)

In [147]:
#filters the dataframe to only include rows containing the string "pbs.twimg.com/media"
tweet_df = tweet_df[tweet_df['tweet_img_URL'].str.contains("pbs.twimg.com/media")]

tweet_df.head(10)

,tweet_id,tweet_txt,created_at,tweet_img_URL
0,1376469740949008391,"I am tired of it, so I am posting this, Megan Markle claiming she never did any research about the Royal Family in Uk, yet in past she mentioned in an interview, that as a teenager she went to UK, took a picture of herself afront Buckingham Palace, someone has amnesia but I don't https://t.co/Ki0Ya96iJv",2021-03-29 09:42:14,https://pbs.twimg.com/media/ExozlCHXEAQE_Mt.jpg
2,1376440248570421250,@JeanTrust You can see the hate and jealously in this photo. Megan looks like she could have killed her in this photo. https://t.co/Vqd4sDgwd8,2021-03-29 07:45:02,https://pbs.twimg.com/media/ExoYw3DWQAIHUX3.jpg
3,1376349957813370882,(GSN) Prince Harry and Meghan Markle’s royal exit is being turned into a movie. Lifetime has announced the next installment in its Harry &amp; Megan franchise titled “Harry &amp; Meghan: Escaping the Palace.” https://t.co/SqAinZCdpH https://t.co/oo9eR7sFyI,2021-03-29 01:46:15,https://pbs.twimg.com/media/ExnG_gpWUAgat_3.jpg
4,1376323374658490373,"#Headlines: Tom Holland celebrates last day of Spider-Man training with shirtless pic, legendary author Beverly Cleary has died, Shawn Mendes and Camila Cabello's home broken into, Piers Morgan says Megan Markle was treated ""no worse"" than Princess Diana https://t.co/NptSidAYvz https://t.co/gcN7fYbV7M",2021-03-29 00:00:38,https://pbs.twimg.com/media/Exmu0MHXEAMWmZU.jpg
7,1376306117748391937,Megan gets way more hate than Cardi and it’s even worse bc ppl are constantly bringing up a violent assault that she was a victim of. Yet you don’t see her on Twitter everyday replying to trolls. https://t.co/4geQDku9zO,2021-03-28 22:52:03,https://pbs.twimg.com/media/ExmfHdgWUAEhwoP.jpg
8,1376296663107596288,Who is this bitch? Because I know it’s not Megan Fox. https://t.co/hMZGqJALIS,2021-03-28 22:14:29,https://pbs.twimg.com/media/ExmWhPoVgAIdsEM.jpg
10,1376154713524117504,my country talks about meghan markle like come on can we talk about Megan fox https://t.co/s0GnBHTxQW,2021-03-28 12:50:26,https://pbs.twimg.com/media/ExkVaPZU8AEXeFF.png
11,1376150081255129088,Harry's new job sparks more questions around Oprah interview claim - 'It doesn't add up!' https://t.co/iNw9v9AHlW \n\nInteresting fact - resilience 🤔\n\nHarry your first pupil should be Megan 😂\n\n#HarryandMeghan teach #resilience https://t.co/HHiQy4KVu5,2021-03-28 12:32:01,https://pbs.twimg.com/media/ExkRHwbXEAI_q8C.jpg
13,1376129045734305801,EXCELLENT 5 PAGES ON PIERS MORGAN IN TODAYS MAIL ON SUNDAY AND WHY HE THINKS MEGAN MARKLE IS A LYING BITCH! https://t.co/JbkLqyZqvi,2021-03-28 11:08:26,https://pbs.twimg.com/media/Exj-EoEXEAEQveU.jpg
15,1375968423319105537,"Megan: your honor I'm a freak bitch, handcuffs, leashes\nMe: https://t.co/yiCWKONrZp",2021-03-28 00:30:11,https://pbs.twimg.com/media/Exhr_CjXIAYy61q.jpg


In [146]:
#saves the dataframe to .xlsx file without a column for row indexing
tweet_df.to_excel('dataset-1.xlsx', index=False)  